In [ ]:
import warnings, os
import datetime as dt
import itertools as it
import pandas as pd

warnings.filterwarnings('ignore')
pd.set_option('display.width',       None)
pd.set_option('display.max_columns', None)

In [ ]:
import MetaTrader5 as mt5

# Connection

In [ ]:
RICO_REAL   = { 'login':16331371,   'password':'#000000Zz', 'server':'Rico-PRD'                 }
CLEAR_REAL  = { 'login':1018156111, 'password':'#000000Zz', 'server':'ClearInvestimentos-CLEAR' }

PATH_7 = r'C:\Program Files\MetaTrader 5 _7\terminal64.exe'

assert mt5.initialize(PATH_7, **CLEAR_REAL)

# Variables

In [ ]:
SYMBOLS = ['WIN$'] # ['WDO$']
TFRAMES = ['M5','M15','H1','H3','D1','W1']

# Load

In [ ]:
def MT5_LOAD_DATA(SYMBOLS, TFRAMES, start_pos=0, count=99_999):
    Pipe = []
    for (symbol, TF) in it.product(SYMBOLS, TFRAMES):
        timeframe = { 'M1':mt5.TIMEFRAME_M1, 'M5':mt5.TIMEFRAME_M5, 'M10':mt5.TIMEFRAME_M10, 'M15':mt5.TIMEFRAME_M15, 'M20':mt5.TIMEFRAME_M20, 'M30':mt5.TIMEFRAME_M30, 'H1':mt5.TIMEFRAME_H1, 'H2':mt5.TIMEFRAME_H2, 'H3':mt5.TIMEFRAME_H3, 'H4':mt5.TIMEFRAME_H4, 'H6':mt5.TIMEFRAME_H6, 'H8':mt5.TIMEFRAME_H8, 'H12':mt5.TIMEFRAME_H12, 'D1':mt5.TIMEFRAME_D1, 'W1':mt5.TIMEFRAME_W1 }[TF]
        Raw       = pd.DataFrame(mt5.copy_rates_from_pos(symbol, timeframe, start_pos, count))
        Raw.insert(0, 'a',          Raw.index + 1)
        Raw.insert(1, 'z',          Raw.index - Raw.index.max())
        Raw.insert(2, 'symbol',     symbol)
        Raw.insert(3, 'tf',         TF)
        Raw.insert(4, 'datetime',   pd.to_datetime(Raw['time'], unit='s'))
        Pipe.append(Raw)
    pass
    return pd.concat(Pipe, ignore_index=1)

Raw = MT5_LOAD_DATA(SYMBOLS, TFRAMES)
Raw.head()

In [ ]:
raise Exception('STOP')

In [ ]:
Raw.to_csv('Storage/Raw.csv', index=False)
Raw = pd.read_csv('Storage/Raw.csv', parse_dates=['datetime'])
Raw.head()

In [ ]:
def QUERY_SELECT(Raw, SYMBOLS, TFRAMES):
    return Raw[Raw['symbol'].isin(SYMBOLS) & Raw['tf'].isin(TFRAMES)] .reset_index(drop=1)

Query = QUERY_SELECT(Raw, SYMBOLS=['WIN$'], TFRAMES=['D1'])
Query.head()